In [1]:
import numpy as np

In [2]:
"""
目标为寻找使f(x,y) = x^2 + y^2(x∈{1,2,3,4,5,6,7},y∈{1,2,3,4,5,6,7})
取得最大值的x,y

产生编码(x,y)一对x,y可以变为一个长为y的list，每个数字为0 or 1
"""


'\n目标为寻找使f(x,y) = x^2 + y^2(x∈{1,2,3,4,5,6,7},y∈{1,2,3,4,5,6,7})\n取得最大值的x,y\n\n产生编码(x,y)一对x,y可以变为一个长为y的list，每个数字为0 or 1\n'

In [3]:
def train(M = 5):

    X = []
    for i in range(M):
        x1 = np.random.randint(1,8)
        x2 = np.random.randint(1,8)
        X.append((x1, x2))

    for i in range(M):
        X[i] = encoding(X[i][0], X[i][1])
    return X

In [4]:
def encoding(num1, num2):
    code1 = list(bin(num1)[2:])
    code2 = list(bin(num2)[2:])
    # 补足位数
    while(len(code1)) < 3:
        code1 = ['0'] + code1
    while(len(code2)) < 3:
        code2 = ['0'] + code2       
        
    for i in range(len(code1)):
        code1[i] = eval(code1[i])

    for i in range(len(code2)):
        code2[i] = eval(code2[i])
        
    code = code1 + code2
    return code

In [5]:
def decoding(code):
    halflen = len(code)//2
    code1 = code[:halflen]
    code2 = code[halflen:]
    str1 = str2 = ""
    
    for i in range(halflen):
        str1 += str(code1[i])
        str2 += str(code2[i])
    num1 = int(str1, 2) # 后一个参数说明数字原来为什么进制
    num2 = int(str2, 2)
    return num1, num2

In [6]:
def fitness(num1, num2):
    return (num1**2 + num2**2)

In [7]:
def RMS(example_list, copy_times = 5):

    # times 为转轮盘的次数
    # 输入的为已编码的列表
    M = len(example_list)
    fitness_list = []
    # list中每一个元素应该为[1,0,1,0,1,1]这种类型
    for example in example_list:
        num1, num2 = decoding(example)
        fitness_per = fitness(num1, num2)
        fitness_list.append(fitness_per)
    #计算概率
    
    fitness_list = [s/sum(fitness_list) for s in fitness_list]
    
    copy_list = []
    
    for i in range(copy_times):
        pro_tmp = 0 # 产生的pro在pro_tmp 和pro_sum + fitness_list[j]之间算作抽到了j
        pro = np.random.random()

        for j in range(M):
            pro_tmp += fitness_list[j] # pro_tmp是逐渐变大的
            if pro <= pro_tmp:
                copy_list.append(example_list[j])
                break
                
    return copy_list

In [8]:
X = train(M = 100)


In [9]:
def crossover(code1, code2, p_crossover = 0.85):

    codelen = len(code1)
    pro = np.random.random()
    if pro < p_crossover:
        crosspoint = np.random.randint(codelen)
        code1_front = code1[:crosspoint]
        code1_back = code1[crosspoint:]
        code2_front = code2[:crosspoint]
        code2_back = code2[crosspoint:]
        code1_new = code1_front + code2_back
        code2_new = code2_front + code1_back
    else:
        code1_new, code2_new = code1, code2
        
    return code1_new, code2_new

In [10]:
def mutation(code, p_variation = 0.1):

    codelen = len(code)
    pro = np.random.random() 
    if pro < p_variation:
        varpoint = np.random.randint(codelen)
        code[varpoint] = 1 - code[varpoint]
    return code

In [12]:
def GA(X, elite_select = False, generations = 5, copy_times = 6):
    # copy_times 交配次数
    parents = X
    for generation in range(generations):        
        parents_selected = RMS(parents, copy_times = copy_times)
        # 精英选择
        # 在每一次产生新的一代时，首先把当前最优解原封不动的复制到新的一代中。
        if elite_select == True:
            elite_dict = {}
            parents_fitness = np.array([fitness(x[0], x[1]) for x in [decoding(c) for c in parents_selected]])
            elite_list = (parents_fitness == np.max(parents_fitness)) # bool类型的list

            for t in range(len(elite_list)):
                if elite_list[t] == True:
                    elite_dict[t] = parents_selected[t] # 当前最优秀的个体dic
   
        
        # 交叉
        for j in range(copy_times//2):
            parents_selected[2*j], parents_selected[2*j+1] = \
            crossover(parents_selected[2*j], parents_selected[2*j+1])
        #变异 
        for j in range(copy_times):
            parents_selected[j] = mutation(parents_selected[j])
            
        fitness_list = [fitness(x[0], x[1]) for x in [decoding(c) for c in parents_selected]]
        # print(fitness_list)
        parents = parents_selected
        
        if elite_select == True:
            for key, value in elite_dict.items():                
                if fitness(decoding(parents[key])[0],decoding(parents[key])[1]) < \
                    fitness(decoding(elite_dict[key])[0],decoding(elite_dict[key])[1]):
                    parents[key] = elite_dict[key]

        if generation % 1 == 0:
            print([decoding(x) for x in parents])
            
    parents_decoding = [decoding(x) for x in parents]
    return parents_decoding

In [13]:
X = train(M = 100)
GA(X, elite_select = True, generations = 5, copy_times = 50)

[(2, 2), (7, 7), (1, 5), (2, 7), (6, 3), (6, 5), (5, 6), (6, 7), (7, 7), (5, 1), (5, 7), (6, 6), (5, 7), (6, 2), (6, 1), (2, 7), (6, 6), (6, 5), (5, 1), (4, 6), (7, 5), (1, 7), (7, 6), (6, 7), (7, 6), (6, 3), (1, 7), (1, 5), (5, 6), (6, 6), (7, 5), (1, 5), (6, 3), (7, 7), (7, 0), (7, 6), (4, 6), (7, 2), (4, 4), (6, 6), (7, 1), (2, 5), (6, 7), (5, 7), (6, 7), (6, 7), (6, 1), (7, 4), (7, 5), (7, 5)]
[(6, 3), (6, 7), (5, 7), (6, 5), (7, 4), (6, 5), (7, 7), (7, 7), (7, 7), (2, 7), (6, 5), (5, 3), (7, 7), (7, 7), (6, 6), (6, 7), (7, 6), (6, 6), (6, 7), (5, 7), (7, 7), (5, 4), (7, 7), (7, 3), (7, 6), (7, 6), (7, 7), (6, 7), (7, 5), (7, 7), (7, 6), (5, 6), (7, 7), (7, 7), (7, 7), (6, 5), (7, 7), (7, 7), (5, 2), (7, 7), (1, 5), (6, 7), (6, 6), (7, 3), (5, 0), (7, 6), (2, 7), (7, 7), (6, 7), (5, 7)]
[(3, 0), (4, 7), (7, 7), (7, 7), (7, 7), (7, 7), (7, 7), (7, 7), (7, 7), (7, 4), (7, 7), (7, 7), (7, 7), (7, 7), (7, 7), (4, 7), (6, 6), (5, 7), (7, 7), (3, 7), (7, 7), (7, 7), (7, 7), (7, 7), (6, 6

[(3, 7),
 (7, 7),
 (7, 7),
 (7, 7),
 (7, 7),
 (7, 7),
 (7, 7),
 (7, 7),
 (7, 7),
 (6, 7),
 (7, 7),
 (7, 7),
 (7, 7),
 (7, 7),
 (7, 7),
 (7, 7),
 (7, 7),
 (7, 7),
 (7, 7),
 (4, 7),
 (7, 7),
 (7, 7),
 (7, 7),
 (7, 7),
 (7, 7),
 (7, 7),
 (7, 7),
 (7, 7),
 (7, 7),
 (7, 7),
 (4, 7),
 (7, 7),
 (7, 4),
 (7, 7),
 (7, 7),
 (4, 5),
 (7, 7),
 (7, 7),
 (7, 7),
 (4, 7),
 (7, 7),
 (7, 7),
 (7, 7),
 (7, 7),
 (7, 7),
 (7, 7),
 (7, 7),
 (7, 7),
 (4, 7),
 (7, 7)]

In [14]:
Y = train(M = 100)
GA(Y, generations = 5, copy_times = 50)

[(6, 7), (7, 3), (5, 7), (2, 3), (4, 7), (1, 4), (4, 5), (5, 7), (4, 4), (7, 5), (6, 4), (6, 7), (0, 2), (6, 0), (6, 3), (2, 3), (2, 7), (4, 5), (2, 7), (5, 3), (5, 3), (6, 4), (4, 5), (6, 1), (4, 2), (4, 5), (7, 5), (6, 5), (6, 5), (6, 7), (6, 3), (1, 5), (6, 6), (1, 5), (4, 5), (4, 2), (5, 4), (4, 5), (7, 3), (2, 6), (6, 4), (6, 4), (1, 7), (6, 1), (7, 6), (1, 6), (3, 0), (6, 5), (5, 3), (1, 3)]
[(6, 5), (6, 5), (6, 3), (4, 7), (5, 2), (2, 1), (6, 2), (4, 1), (1, 7), (6, 7), (2, 4), (6, 2), (6, 4), (7, 5), (6, 7), (5, 7), (6, 0), (7, 5), (5, 3), (6, 4), (5, 4), (6, 7), (6, 4), (6, 0), (5, 3), (6, 4), (6, 5), (4, 5), (4, 2), (7, 6), (7, 2), (2, 7), (6, 5), (6, 5), (5, 7), (2, 3), (5, 3), (6, 7), (4, 4), (4, 5), (7, 3), (4, 7), (7, 3), (2, 3), (5, 7), (6, 4), (6, 4), (7, 3), (6, 7), (1, 7)]
[(6, 4), (6, 5), (5, 3), (7, 3), (7, 3), (5, 2), (6, 4), (6, 7), (4, 5), (4, 2), (7, 7), (6, 6), (5, 4), (6, 5), (6, 5), (6, 7), (2, 7), (1, 7), (1, 3), (6, 7), (1, 4), (5, 3), (5, 7), (6, 4), (5, 6

[(6, 7),
 (5, 7),
 (6, 7),
 (6, 5),
 (5, 6),
 (5, 1),
 (6, 7),
 (6, 6),
 (5, 2),
 (6, 7),
 (6, 6),
 (7, 7),
 (7, 7),
 (5, 5),
 (7, 3),
 (6, 6),
 (6, 6),
 (6, 7),
 (7, 3),
 (6, 7),
 (7, 7),
 (7, 3),
 (5, 3),
 (4, 7),
 (6, 5),
 (5, 7),
 (2, 7),
 (5, 7),
 (6, 5),
 (5, 7),
 (6, 7),
 (2, 7),
 (6, 7),
 (6, 4),
 (1, 3),
 (5, 7),
 (7, 7),
 (7, 5),
 (5, 5),
 (5, 7),
 (0, 7),
 (6, 5),
 (5, 5),
 (4, 7),
 (1, 7),
 (2, 7),
 (6, 7),
 (4, 7),
 (7, 3),
 (7, 5)]